In [7]:
import pandas as pd 
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [8]:
# Download the VADER lexicon for sentiment analysis if not already present.
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\mahmo\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [17]:

def fetch_data_form_sql():
    conn_str = (
        "Driver={SQL Server};"
        "Server=DESKTOP-UMGLMNG\\SQLEXPRESS;"
        "Database=marketing_analytics;"
        "Trusted_Connection=yes;"
    )
    conn = pyodbc.connect(conn_str)
    query = """
        SELECT ReviewID, CustomerID, ProductID, Rating, ReviewText
        FROM fact_customer_reviews
    """
    df = pd.read_sql(query, conn)
    conn.close()
    return df


customer_reviews_df = fetch_data_form_sql()

sia = SentimentIntensityAnalyzer()


def calculate_sentiment(review):
    sentiment = sia.polarity_scores(review)
    return sentiment['compound']


def categorize_sentiment(score, rating):
    if score > 0.05:
        if rating >= 4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed Positive'
        else:
            return 'Mixed Negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed Negative'
        else:
            return 'Mixed Positive'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <= 2:
            return 'Negative'
        else:
            return 'Neutral'


def sentiment_bucket(score):
    if score >= 0.5:
        return '0.5 to 1.0'
    elif 0.0 <= score < 0.5:
        return '0.0 to 0.49'
    elif -0.5 <= score < 0.0:
        return '-0.49 to 0.0'
    else:
        return '-1.0 to -0.5'


# Calculate sentiment score
customer_reviews_df['SentimentScore'] = customer_reviews_df['ReviewText'].apply(calculate_sentiment)

# Category based on score + rating
customer_reviews_df['SentimentCategory'] = customer_reviews_df.apply(
    lambda row: categorize_sentiment(row['SentimentScore'], row['Rating']),
    axis=1
)

# Bucket ranges
customer_reviews_df['SentimentBucket'] = customer_reviews_df['SentimentScore'].apply(sentiment_bucket)

print(customer_reviews_df.head())

customer_reviews_df.to_csv('fact_customer_reviews_with_sentiment.csv', index = False)

C:\Users\mahmo\AppData\Local\Temp\ipykernel_10864\276692343.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


   ReviewID  CustomerID  ProductID  ...  SentimentScore SentimentCategory  SentimentBucket
0         1          77         18  ...         -0.3089    Mixed Negative     -0.49 to 0.0
1         2          80         19  ...          0.0000          Positive      0.0 to 0.49
2         3          50         13  ...          0.0000          Positive      0.0 to 0.49
3         4          78         15  ...          0.2382    Mixed Positive      0.0 to 0.49
4         5          64          2  ...         -0.3089    Mixed Negative     -0.49 to 0.0

[5 rows x 8 columns]
